# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234803432364                   -0.50    8.0         
  2   -7.250267340547       -1.81       -1.40    1.0   9.04ms
  3   -7.251165033515       -3.05       -2.15    1.0   9.12ms
  4   -7.251254673013       -4.05       -2.16    2.0   11.8ms
  5   -7.251328383382       -4.13       -2.60    1.0   9.48ms
  6   -7.251338332297       -5.00       -3.22    1.0   9.32ms
  7   -7.251338745063       -6.38       -3.71    1.0   68.6ms
  8   -7.251338783533       -7.41       -3.96    2.0   11.4ms
  9   -7.251338797311       -7.86       -4.53    1.0   10.0ms
 10   -7.251338798599       -8.89       -5.09    2.0   11.3ms
 11   -7.251338798686      -10.06       -5.48    2.0   11.5ms
 12   -7.251338798703      -10.78       -6.00    2.0   11.5ms
 13   -7.251338798704      -11.84       -6.55    2.0   11.9ms
 14   -7.251338798705      -13.33       -6.97    1.0   10.0ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.037511481488360365
[ Info: Arnoldi iteration step 2: normres = 0.7895533966696276
[ Info: Arnoldi iteration step 3: normres = 0.6386859738692842
[ Info: Arnoldi iteration step 4: normres = 0.31733755409981645
[ Info: Arnoldi iteration step 5: normres = 0.1691806296998378
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (6.82e-03, 3.76e-02, 1.16e-01, 1.10e-01, 3.93e-02)
[ Info: Arnoldi iteration step 6: normres = 0.6498646394636235
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.59e-03, 3.71e-02, 5.97e-01, 1.96e-01, 5.22e-02)
[ Info: Arnoldi iteration step 7: normres = 0.13956743083308493
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.02e-04, 1.06e-01, 2.39e-02, 4.23e-02, 7.19e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11340817140921357
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.52e